In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re


In [2]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "dubai_abu-dhabi.osm"  # Replace this with your osm file
SAMPLE_FILE = "dubai_sample.osm"

k = 1 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [3]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

from collections import defaultdict
osm_file = open("dubai_abu-dhabi.osm", "r")




def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=("start",)):
        t=[]
        if elem.tag:
            t.append(elem.tag)           
            for elem.tag in t:
                if elem.tag in tags.keys():
                    tags[elem.tag] += 1 
                else:
                    tags[elem.tag] = 1
    print tags
count_tags(osm_file)


IndexError: pop from empty stack

In [6]:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("dubai_abu-dhabi.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    


audit()


IndexError: pop from empty stack

In [18]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
osm_file = open("dubai_sample.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file,events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])    
    pprint.pprint(dict(street_types))    


audit()


{'12': 1,
 '147': 1,
 '18': 1,
 '2': 3,
 '50': 1,
 '74147': 1,
 'Arab': 3,
 'Avenue': 5,
 'Circuit': 1,
 'Dubai': 1,
 'East': 1,
 'Hill': 1,
 'Lane': 5,
 'Masyaf': 3,
 'Place': 10,
 'Road': 25,
 'South': 1,
 'St.': 3,
 'Street': 23,
 'Street)': 1,
 'Village': 1,
 'Way': 5,
 'street': 2}


In [25]:

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        k = element.get("k")
        l = re.search(lower,k)
        lc = re.search(lower_colon,k)
        pc = re.search(problemchars,k)
        if l:
            l.group()
            keys['lower'] +=1
        elif lc:
            lc.group()
            keys['lower_colon'] +=1
        elif pc:
            pc.group()
            keys['problemchars'] +=1
        else:
            keys['other'] +=1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys
keys = process_map('dubai_sample.osm')
pprint.pprint(keys)
            

 
 
.
{'lower': 47147, 'lower_colon': 1951, 'other': 295, 'problemchars': 3}


In [28]:
#number of uniques users contributing to the dubai openstreet data
def get_user(element):
    if element.get('uid'):
        uid = element.attrib["uid"]
        return uid
    else:
        return None
    return 


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))   
        pass

    return users

users = process_map('dubai_sample.osm')
len(users)

847

In [33]:
OSMFILE = "dubai_sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Lane", "Way", "Avenue", "Place","Road"]

# UPDATE THIS VARIABLE
mapping = { "street": "Street",
            " St.": "Street",
           "St": "Street",
            "Street)": "Street"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    for search_error in mapping:
        if search_error in name:
            name = re.sub(r'\b' + search_error + r'\b\.?', mapping[search_error], name)
    return name

st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name



{'12': set(['District 12']),
 '12A': set(['12A']),
 '147': set(['147']),
 '18': set(['Street 18']),
 '2': set(['Dubai Investment Park 2', 'Hattan Street 2']),
 '3005': set(['P.O. Box: 3005']),
 '50': set(['P.O. Box 50']),
 '7': set(['Street 7']),
 '74147': set(['P.O.Box 74147']),
 'Arab': set(['Mina Al Arab']),
 'B': set(['Al Hudaiba Rd   21 B']),
 'BITHNAH': set(['BITHNAH']),
 'Circuit': set(['Yas Marina Circuit']),
 'Drive': set(['Yas Leisure Drive']),
 'Dubai': set(['Street 13, Dubai']),
 'E': set(['Road E']),
 'East': set(['Corniche Road East']),
 'GROCERY': set(['AHMED SAEED GROCERY', 'MUSALLAH EID/AL ZAIN GROCERY']),
 'Hail': set(['Abu Hail']),
 'Hill': set(['Newbridge Hill']),
 'Island': set(['Paragon Mall, Reem Island', 'Yas Island']),
 'Khalidiya': set(['Khalidiya']),
 'Mall': set(['Paragon Mall', 'Yas Mall']),
 'Masyaf': set(['Dar Al Masyaf']),
 'Musaffah': set(['Musaffah']),
 'QIRIYA': set(['QIRIYA']),
 'ROAD': set(['AL THAIB ROAD']),
 'Ramth': set(['Al Ramth']),
 'Rd': set(

error: unbalanced parenthesis

In [42]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from pprint import pprint
import copy
import schema
import cerberus

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

pprint(SCHEMA)


{'node': {'schema': {'changeset': {'coerce': <type 'int'>,
                                   'required': True,
                                   'type': 'integer'},
                     'id': {'coerce': <type 'int'>,
                            'required': True,
                            'type': 'integer'},
                     'lat': {'coerce': <type 'float'>,
                             'required': True,
                             'type': 'float'},
                     'lon': {'coerce': <type 'float'>,
                             'required': True,
                             'type': 'float'},
                     'timestamp': {'required': True, 'type': 'string'},
                     'uid': {'coerce': <type 'int'>,
                             'required': True,
                             'type': 'integer'},
                     'user': {'required': True, 'type': 'string'},
                     'version': {'required': True, 'type': 'string'}},
          'type': 'dict'},
 'n

In [43]:

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS,
                  problem_chars = PROBLEMCHARS, default_tag_type = 'regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]
        for child in element:
            tag = {}
            if PROBLEMCHARS.search(child.attrib["k"]):
                continue
        
            elif LOWER_COLON.search(child.attrib["k"]):
                tag_type = child.attrib["k"].split(':',1)[0]
                tag_key = child.attrib["k"].split(':',1)[1]
                tag["key"] = tag_key
                if tag_type:
                    tag["type"] = tag_type
                else:
                    tag["type"] = 'regular'
            
                tag["id"] = element.attrib["id"]
                tag["value"] = child.attrib["v"]
            else:
                tag["value"] = child.attrib["v"]
                tag["key"] = child.attrib["k"]
                tag["type"] = "regular"
                tag["id"] = element.attrib["id"]
            if tag:
                tags.append(tag)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for child in element:
            nd = {}
            tag = {}
            if child.tag == 'tag':
                if PROBLEMCHARS.search(child.attrib["k"]):
                    continue
                elif LOWER_COLON.search(child.attrib["k"]):
                    tag_type = child.attrib["k"].split(':',1)[0]
                    tag_key = child.attrib["k"].split(':',1)[1]
                    tag["key"] = tag_key
                    if tag_type:
                        tag["type"] = tag_type
                    else:
                        tag["type"] = 'regular'
                    tag["id"] = element.attrib["id"]
                    tag["value"] = child.attrib["v"]
    
                else:
                    tag["value"] = child.attrib["v"]
                    tag["key"] = child.attrib["k"]
                    tag["type"] = "regular"
                    tag["id"] = element.attrib["id"]
                if tag:
                    tags.append(tag)
                    
            elif child.tag == 'nd':
                nd['id'] = element.attrib["id"]
                nd['node_id'] = child.attrib["ref"]
                nd['position'] = len(way_nodes)
            
                if nd:
                    way_nodes.append(nd)
            else:
                continue
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [44]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


osm_sample = "dubai_abu-dhabi_sample.osm"
                    
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(osm_sample, validate = False)

In [10]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'nodes_tag.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(sqlite_file)
conn.text_factory = str
cur = conn.cursor()
# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tag(id INTEGER, key TEXT, value TEXT,type TEXT)
''')
# commit the changes
conn.commit()
# Read in the csv file as a dictionary, format the
# data as a list of tuples:


In [11]:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO nodes_tag(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()
cur.execute('SELECT * FROM nodes_tag')
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)
conn.close()


1):
[(31248042, 'source', 'Bing', 'regular'),
 (31309316, 'source', 'Bing', 'regular'),
 (31309326, 'source', 'Bing', 'regular'),
 (31309336, 'source', 'Bing', 'regular'),
 (31309346, 'source', 'Bing', 'regular'),
 (31309361, 'source', 'Bing', 'regular'),
 (31309783, 'source', 'Bing', 'regular'),
 (31309793, 'source', 'Bing', 'regular'),
 (31309803, 'source', 'Bing', 'regular'),
 (31309813, 'source', 'Bing', 'regular'),
 (31310353, 'source', 'Bing', 'regular'),
 (31310367, 'source', 'Bing', 'regular'),
 (31310378, 'source', 'Bing', 'regular'),
 (31475449, 'source', 'Bing', 'regular'),
 (31475460, 'source', 'Bing', 'regular'),
 (31475470, 'source', 'Bing', 'regular'),
 (31475480, 'source', 'Bing', 'regular'),
 (83748119, 'source', 'Bing', 'regular'),
 (86063874, 'source', 'Bing', 'regular'),
 (87268089, 'source', 'Bing', 'regular'),
 (87268103, 'source', 'Bing', 'regular'),
 (87268117, 'source', 'Bing', 'regular'),
 (87268129, 'source', 'Bing', 'regular'),
 (87268146, 'source', 'Bing', 

In [13]:
string=\xd8\xa8\xd9\x86\xd9\x83 \xd8\xa7\xd8\xa8\xd9\x88\xd8\xb8\xd8\xa8\xd9\x8a \xd8\xa7\xd9\x84\xd8\xaa\xd8\xac\xd8\xa7\xd8\xb1\xd9\x8a      ADCB 

print string

SyntaxError: unexpected character after line continuation character (<ipython-input-13-cc3aa014cc61>, line 1)